In [5]:
import pickle
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
#from hpsklearn import HyperoptEstimator , tfidf, pca, any_text_preprocessing
#from hyperopt import hp, tpe, fmin
#from hyperopt import STATUS_OK, Trials
#from hyperopt import space_eval
from sklearn.metrics import accuracy_score


In [6]:
with open('NHlegoverall.dict','rb') as f:
    leg_data = pickle.load(f)
with open('NHlegoverallLabel.list','rb') as f:
    leg_target = pickle.load(f)    
#print(leg_data.keys())

In [7]:
#lets try including the document embeddings
doc_embeddings = pd.read_pickle("NHlegtext_embed_wBill.DF")
#print(doc_embeddings)
#doc_embeddings.describe()

In [8]:
#seems like we dont actually have the correct document embeddings yet
#print(doc_embeddings['bill_id'][0:5])
#print(doc_embeddings['bill_id'][-5:-1])

In [9]:
#preprocess the data into just the most useful bits
#i think we might want to reduce the number of bill categories
#print(leg_data.keys())

#only include and train on bill_cat, chamber, sponsor_vec2, time_t
new_leg_data = {'bill_cat':leg_data['bill_cat'], 'chamber':leg_data['chamber'], 'sponsor_vec2':leg_data['sponsor_vec2'], 'time_t':leg_data['time_t']}

#normalize the time data
for i, value in enumerate(new_leg_data['time_t']):
    new_leg_data['time_t'][i] = value/10
    



In [10]:
#set the default embedding to the average embedding from df.describe()
default_embedding = {'doc_id':0, 'bill_id':0, 'dv1':-0.429835,'dv2':-0.348652,'dv3':-0.096175,'dv4':0.722217,'dv5':-0.080119,'dv6':0.331679,'dv7':0.886933,'dv8':0.934954,'dv9':-0.798512,'dv10':0.054533,'dv11':-0.028231,'dv12':0.561587,'dv13':-0.485424,'dv14':0.015666,'dv15':0.864729,'dv16':-0.697787}

In [11]:
#######This didnt work, so we will need actual text features rather than the embedding
doc_dict = doc_embeddings.to_dict('list')
print(doc_dict.keys())

#here, we need to make add the correct embedding to the correct row of the leg_data
# do this with two for loops (gonna try in the morning)

new_doc_dict = {}
for column in doc_dict:
    new_doc_dict[column] = []
    for bill_id in leg_data['bill_id']:
        try:
            bill_id_position = doc_dict['bill_id'].index(str(bill_id))
            new_doc_dict[column].append(doc_dict[column][bill_id_position])
        except:
            new_doc_dict[column].append(default_embedding[column])
            
del new_doc_dict['doc_id']
del new_doc_dict['bill_id']
print(new_doc_dict.keys())



#didn't help
#new_leg_data = {**new_leg_data, **new_doc_dict}

dict_keys(['doc_id', 'dv1', 'dv2', 'dv3', 'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9', 'dv10', 'dv11', 'dv12', 'dv13', 'dv14', 'dv15', 'dv16', 'bill_id'])
dict_keys(['dv1', 'dv2', 'dv3', 'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9', 'dv10', 'dv11', 'dv12', 'dv13', 'dv14', 'dv15', 'dv16'])


In [12]:
leg_data = new_leg_data

print(leg_data.keys())
print(leg_data['time_t'][0:10])

dict_keys(['bill_cat', 'chamber', 'sponsor_vec2', 'time_t'])
[0.1, 0.2, 0.3, 0.4, 0.5, 0.1, 0.2, 0.3, 0.4, 0.5]


In [13]:
leg_data_df = pd.DataFrame(leg_data)
leg_data_df = pd.get_dummies(leg_data_df)
leg_label_df = pd.DataFrame(leg_target)
leg_label_df = pd.get_dummies(leg_label_df)


In [14]:
X_train, X_test, y_train, y_test = train_test_split (leg_data_df, leg_target, test_size=0.3, random_state=109, shuffle=False)

print(X_train.values)

[[1.         0.1        0.         ... 0.         1.         0.        ]
 [1.         0.2        0.         ... 0.         1.         0.        ]
 [1.         0.3        0.         ... 0.         1.         0.        ]
 ...
 [0.88888889 1.8        0.         ... 0.         1.         0.        ]
 [0.88888889 1.9        0.         ... 0.         1.         0.        ]
 [0.88888889 2.         0.         ... 0.         1.         0.        ]]


In [15]:
#This frame balances the testing data so that it is 50% pass and 50% fail

#Also shuffle the X_train and X_test data (make sure they are shuffled in exactly the same order as each other)

y_test = np.array(y_test)
passed_bool = y_test == 1
unpassed = X_test[y_test == 0]

sampled_X = X_test.sample(frac=0.124)
print(X_test)




############This code balances the test data to 50/50 pass/fail

mask = []

for i, value in enumerate(y_test):
    if value == 1:
        mask.append(True)
    if value == 0: 
#         #gives about 0.125 of the false samples
        if i%8 == 0:
             mask.append(True)
        else:
             mask.append(False)
            
y_test = list(np.array(y_test)[mask])
X_test = X_test[mask]
   
# print(len(y_test))
# X_test.describe()


       sponsor_vec2  time_t  bill_cat_0  bill_cat_1  bill_cat_2  bill_cat_3  \
23809         0.125     0.1           0           0           0           0   
23810         0.125     0.2           0           0           0           0   
23811         0.125     0.3           0           0           0           0   
23812         0.125     0.4           0           0           0           0   
23813         0.125     0.5           0           0           0           0   
...             ...     ...         ...         ...         ...         ...   
34009         0.500     0.1           0           0           0           0   
34010         0.500     0.1           0           0           0           0   
34011         0.500     0.1           0           0           0           0   
34012         0.500     0.1           0           0           0           0   
34013         0.000     0.1           0           0           0           0   

       bill_cat_4  bill_cat_5  bill_cat_6  bill_cat

In [16]:
#define a performance metrics function (accuracy and percision and recall)
def perf_metric (y_test, y_pred):
    length = len(y_test)
#Accuracy
    x=0
    y=0
    acc=0
    for i in range(length):
        if y_test[i] == y_pred[i]:
            x+=1
        y+=1
    acc=x/y*100
    print(x,y,len(y_pred),len(y_test))
    print('Accuracy:', acc)

#Precision and Recall
    tp=0
    fp=0
    fn=0
    tn=0
    perc=0 
    recall= 0 
        
    for i in range(length):
        if y_pred[i]==1 and y_test[i]==1:
            tp+=1
        elif y_pred[i]==1 and y_test[i]==0:
            fp+=1
        elif y_pred[i]==0 and y_test[i]==1:
            fn+=1
        elif y_pred[i]==0 and y_test[i]==0:
            tn+=1
        
    if tp==0 and fp==0: 
        perc=0
    else:
        perc= tp/(tp+fp)  
          
    if tp==0 and fn==0: 
        recall=0
    else: 
        recall = tp/(tp+fn)

#print(tp,fp,fn,tn)
    print('Precision:', perc)
    print('Recall', recall)
#AUC Score
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    print('AUC:', roc_auc)

In [17]:
# Performance metrics function 2 (AUC and Cross entropy loss)
def perf_metric2 (y_test, y_pred, y_proba):
#Cross Entropy Loss
    log_loss(y_test, y_proba, eps=1e-15,normalize=True, sample_weight=None, labels=None)
    log_results1=[]
    for i in range(len(y_proba)):
        log_results1.append (log_loss(y_test, y_proba, eps=1e-15,normalize=True, sample_weight=None, labels=None))
    print ('Cross Entropy Loss:', log_results[0])


In [18]:
# define the reporting function for Hyperparameter tuning
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [19]:
##################################### Decision Tree #################################

In [20]:
#Create a Gaussian Classifier
tree = DecisionTreeClassifier()#criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

# specify parameters and distributions to sample from      
param_dist = {"max_depth": [1,2,4,8, 16, 32, 64, 128,256,512],
              "min_samples_split": [2,4,8, 16, 32, 64, 128,256,512],
              "max_features": [1,2,4,8, 16, 27],
              "max_leaf_nodes":[2,4,8, 16, 32, 64, 128, 256,512],
              "splitter":['best', 'random'],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 50
random_search = RandomizedSearchCV(tree, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5, iid=False)
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
     " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

RandomizedSearchCV took 7.29 seconds for 50 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.688 (std: 0.021)
Parameters: {'splitter': 'best', 'min_samples_split': 512, 'max_leaf_nodes': 128, 'max_features': 8, 'max_depth': 128, 'criterion': 'gini'}

Model with rank: 2
Mean validation score: 0.688 (std: 0.036)
Parameters: {'splitter': 'random', 'min_samples_split': 64, 'max_leaf_nodes': 64, 'max_features': 8, 'max_depth': 256, 'criterion': 'entropy'}

Model with rank: 3
Mean validation score: 0.682 (std: 0.029)
Parameters: {'splitter': 'random', 'min_samples_split': 32, 'max_leaf_nodes': 256, 'max_features': 16, 'max_depth': 512, 'criterion': 'entropy'}



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [21]:
#Grid_search 
#sample_split_range = list(range(2, 50))
#param_grid = dict(min_samples_split=sample_split_range)

param_grid = {"max_depth":range(10,11),#range(7,17)
              "min_samples_split":range(60,61), #range(54,64)
              "max_features": range(20,21), #range(15,28)
              "max_leaf_nodes":range(255,256), #range(250,260)
              "splitter":['best', 'random'],
              "criterion":["gini", "entropy"]}

grid_search = GridSearchCV(tree, param_grid=param_grid, cv=5, iid=False)

start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

GridSearchCV took 0.83 seconds for 4 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.678 (std: 0.028)
Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 20, 'max_leaf_nodes': 255, 'min_samples_split': 60, 'splitter': 'best'}

Model with rank: 2
Mean validation score: 0.678 (std: 0.040)
Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 20, 'max_leaf_nodes': 255, 'min_samples_split': 60, 'splitter': 'random'}

Model with rank: 3
Mean validation score: 0.667 (std: 0.030)
Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 20, 'max_leaf_nodes': 255, 'min_samples_split': 60, 'splitter': 'best'}



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [22]:
#estim.fit(X_train.values, y_train.values)
debug_X = pd.DataFrame({'x':[0, 1, 2, 3, 4, 1, 2, 3, 4],
                        'z':[0, 1, 2, 3, 4, 4, 3, 2, 1]})
#rule is if x or y is greater than 4, its true
debug_y = [0,0,0,0,1,1,0,0,1]

estim.predict(debug_X)# pd.DataFrame({'x':[4,2],
                      #      'z':[0,2]}))
    
#y_proba = estim.predict_proba(X_test)
####y_proba_log = tree.predict_log_proba(X_test)

NameError: name 'estim' is not defined

In [ ]:
clf = DecisionTreeClassifier()

mix_algo = partial(mix.suggest, p_suggest=[
(0.05, rand.suggest),
(0.75, tpe.suggest),
(0.20, anneal.suggest)])

#  preprocessing= any_text_preprocessing
estim = HyperoptEstimator(classifier=clf, algo=mix_algo)
estim.fit(X_train.values, y_train)#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)

In [ ]:
perf_metric(y_test,y_pred)

In [ ]:
##################################### Gaussian Naive Bayes #################################

In [ ]:
clf=GaussianNB()

mix_algo = partial(mix.suggest, p_suggest=[
(0.05, rand.suggest),
(0.75, tpe.suggest),
(0.20, anneal.suggest)])


estim = HyperoptEstimator(classifier=clf, algo=mix_algo, max_evals=15)
estim.fit(X_train.values, y_train)#X_train.values, y_train
#clf.fit(X_train,y_train)
y_pred = estim.predict(X_test)
#y_proba = clf.predict_proba(X_test)

In [ ]:
perf_metric(y_test,y_pred)

In [ ]:
perf_metric2(y_test,y_pred,y_proba)

In [ ]:
##################################### Multinomial Naive Bayes #################################

In [ ]:
mnnb = MultinomialNB()
# specify parameters and distributions to sample from
          
param_grid = {}

grid_search = GridSearchCV(mnnb, param_grid=param_grid, cv=5, iid=False)

start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

In [ ]:
clf = MultinomialNB()

estim = HyperoptEstimator(classifier=clf, algo=mix_algo, max_evals=2)
estim.fit(X_train.values, y_train)#X_train.values, y_train
y_pred = estim.predict(X_test)

#Train the model using the training sets
#clf.fit(X_train, y_train)

#Predict the response for test dataset
#y_pred = clf.predict(X_test)
#y_proba = mnnb.predict_proba(X_test)
#y_proba_log = mnnb.predict_log_proba(X_test)

In [ ]:
perf_metric(y_test,y_pred)

In [ ]:
perf_metric(y_test,y_pred,y_proba)

In [ ]:
##################################### Random Forest #################################

In [ ]:
rf = RandomForestClassifier()#criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

# specify parameters and distributions to sample from
          
param_dist = {"max_depth": [16,32,64,128,256],
              "max_features": [2,4,6,8,16],
              "min_samples_split": [16,32,64,128,256],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

param_dist2 = {"criterion"         : ["gini", "entropy"],
               "max_features"      : [0.2,0.5,0.8],
               "max_depth"         : [4,6,10],
               "min_samples_split" : [2, 5, 20,50]}

# run randomized search
n_iter_search = 40
random_search = RandomizedSearchCV(rf, param_distributions=param_dist2,
                                   n_iter=n_iter_search, cv=5, iid=False)
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
     " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [ ]:
RF=RandomForestClassifier()
grid_search = GridSearchCV(RF, param_dist2, n_jobs=-1, cv=5)
grid_search.fit(X_train, y_train)
start = time()
grid_search.fit(X_train, y_train)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

In [ ]:
clf = RandomForestClassifier(
    min_samples_split=5, 
    criterion = 'gini', 
    max_features= 0.2, 
    max_depth= 10)

#y_pred=grid_search.pred(X_test)
#y_pred=grid_search.pred(X_test)


clf = RandomForestClassifier()
clf.fit(X_train, y_train)  
y_pred= clf.predict(X_test)

perf_metric(y_test, y_pred)

In [ ]:
perf_metric2(y_test, y_pred, y_proba)

In [ ]:
##################################### Neural Network #################################

In [ ]:
clf = MLPClassifier()
clf.fit(X_train, y_train)                         

y_pred=clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [ ]:
clf = MLPClassifier()
estim = HyperoptEstimator(classifier=clf, algo=tpe.suggest, max_evals=3)

estim.fit(X_train.values, y_train)

y_pred = estim.predict(X_test)

In [ ]:
perf_metric(y_test, y_pred)

In [ ]:
perf_metric2(y_test,y_pred)